In [2]:
# Импортирую необходимые библиотеки
import pandas as pd 
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("data/vehicles_dataset_upd6.csv")
df.head(5)

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,is_jeep,is_honda,is_nissan,x0_diesel,x0_electric,x0_gas,x0_hybrid,x0_other,std_scaled_odometer,std_scaled_price
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0,0,0,1.0,0.0,0.0,0.0,0.0,-1.079390,2.958509
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.560115,-0.085826
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0,0,0,0.0,0.0,1.0,0.0,0.0,-1.292863,1.406256
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.352621,-0.281218
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.234311,-0.274017


In [4]:
# начинаю готовить данные для модели, чтоб убрать те, что одинаковые, но не совпадают из-за описания
print(df.model.unique())
print(df.model.nunique())

['2500 crew cab big horn' 'explorer 4wd 4dr xlt' 'golf r hatchback' ...
 'gs350' '1988 Nisan' 'a3 2.0t premium plus pzev']
3341


In [5]:
df["short_model"] = df["model"].apply(lambda x: x.split()[0])
print(df["short_model"].unique())
print(df["short_model"].nunique())

['2500' 'explorer' 'golf' 'rav4' '1' 'fusion' 'accord' 'charger' 'altima'
 'galant' '500' 'town' 'e-series' 'g.' 'silverado' 'mustang' 'maxima'
 'civic' 'vue' 'tahoe' 'f-350' 'f150' 'clubman' 'xts' '3500' 'rogue'
 'compass' 'camry' 'renegade' 'forte5' 'f-150' 'wrangler' 'mazda3' '535i'
 'park' 'focus' 'gx' 'titan' 'corolla' 'colorado' 'tiguan' 'impreza'
 'jetta' 'super' 'q5' 'es' 'lx450' 'pilot' 'Diahatsu' 'other' 'sequoia'
 '1500' 'tucson' 'sierra' 'grand' 'nautilus' 'elantra' 'x1' 'camaro'
 'benz' 'tacoma' 'tsx' 'sebring' 'yukon' 'cooper' 'enclave' 'x4' 'sequioa'
 'xc90' 'is300' 'f350' 'romeo' 'ranger' 'spark' '7' 'express' 'q7'
 'tribute' 'xv' 'impala' 'c300' 'frontier' 'fit' 'cc' 'transit' 'outback'
 'passat' '350z' 'optima' 'c10' 'expedition' 'pathfinder' 'challenger'
 '200' 'taurus' 'evoque' 'x3' 'arcadia' 'cr-v' 'xt6' 'sante' 'versa'
 'e150' 'equinox' 'sonata' 'soul' 'regal' 'Scion' 'lucerne' 'f250'
 'odyssey' 's60' 'cruze' 'mkx' 'sienna' '914' '300c' 'crosstrek'
 'Voltswagon' '

Стало меньше уникальных значений, я доволен

Будем нормализововывать категориальные значения

In [6]:
category = ['short_model','transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']
for i in category:
    print(f"{i}:")
    (
        lambda x:print(
            f"nunique : {x.nunique()}\n"
        )
    )(df[i])

short_model:
nunique : 943

transmission:
nunique : 3

region:
nunique : 393

manufacturer:
nunique : 40

state:
nunique : 51

title_status:
nunique : 6

age_category:
nunique : 3



In [8]:
data = df[['short_model','transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']]
data.head()

,short_model,transmission,region,manufacturer,state,title_status,age_category
0,2500,other,chattanooga,ram,tn,clean,new
1,explorer,automatic,north jersey,ford,nj,clean,new
2,golf,other,reno / tahoe,volkswagen,ca,clean,new
3,rav4,automatic,fayetteville,toyota,nc,clean,average
4,1,other,knoxville,bmw,tn,clean,average


In [9]:
#One hot encoder для преобразования категориальных
ohe = OneHotEncoder(sparse_output= False)
ohe.fit(df[['short_model','transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']])
ohe_cat = ohe.transform(df[['short_model','transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']])

In [10]:
ohe.get_feature_names_out()

array(['short_model_-benz', 'short_model_1', 'short_model_124', ...,
       'age_category_average', 'age_category_new', 'age_category_old'],
      dtype=object)

In [11]:
df[ohe.get_feature_names_out()] = ohe_cat
df.head()

C:\Users\proebaldiplom\AppData\Local\Temp\ipykernel_4548\817828308.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()] = ohe_cat
C:\Users\proebaldiplom\AppData\Local\Temp\ipykernel_4548\817828308.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()] = ohe_cat
C:\Users\proebaldiplom\AppData\Local\Temp\ipykernel_4548\817828308.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,state_wy,title_status_clean,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,age_category_average,age_category_new,age_category_old
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
#добавили новые категории, которые будет понимать машинка...
#будем проводить стандартизацию данных
data = df[["lat","long", "year", "odometer/price", "desc_len", "model_in_desc", "model_len"]]

In [15]:
#пробуем через стандартскейлер, потом покажу через минмакс скейлер
std_scaler = StandardScaler()
std_scaler.fit(df[["lat","long", "year", "odometer/price", "desc_len", "model_in_desc", "model_len"]])
std_scaler_col = std_scaler.transform(df[["lat","long", "year", "odometer/price", "desc_len", "model_in_desc", "model_len"]])
std_scaler_col #матрица преобразованных данных(зачем это делается?), при обучении машины данные должны иметь одинаковую размерность, чтоб одна из фичей не влияла больше чем другая просто из-за размерности

array([[-0.61917248,  0.48424452,  1.32239376, ...,  0.63207479,
        -0.15578836,  1.16303238],
       [ 0.38801429,  1.11079989,  0.69597272, ..., -0.64678136,
        -0.15578836,  0.93208734],
       [-0.84305905,  0.531185  ,  0.85257798, ...,  0.56074417,
        -0.15578836,  0.47019727],
       ...,
       [ 1.10542805, -1.62887477, -0.08705357, ..., -0.66825333,
        -0.15578836,  1.50944993],
       [ 0.7739386 ,  0.25195859,  0.53936746, ...,  0.32709999,
        -0.15578836, -0.56905538],
       [-0.33773799,  0.98546471,  0.85257798, ...,  0.63243873,
        -0.15578836,  1.16303238]])

In [16]:
scaled_columns =  ["lat_std", "long_std", "year_std","odometer/price_std", "desc_len_std", "model_in_desc_std", "model_len_std"]

In [ ]:
std_scaled_df = pd.DataFrame(std_scaler_col, columns=scaled_columns)
df = pd.concat([df.reset_index(drop=True), std_scaled_df], axis=1) #добавляем в исходный датасет
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,age_category_average,age_category_new,age_category_old,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,1.0,0.0,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,1.0,0.0,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,1.0,0.0,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,1.0,0.0,0.0,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,1.0,0.0,0.0,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032


In [18]:
#теперь через минмакс скейлер 
from sklearn.preprocessing import MinMaxScaler

In [19]:
dfwithlatlong = df[["lat","long"]]
dfwithlatlong

,lat,long
0,35.060000,-85.250000
1,40.821805,-74.061962
2,33.779214,-84.411811
3,35.715954,-78.655304
4,35.970000,-83.940000
...,...,...
9614,42.123900,-79.189500
9615,43.216990,-77.755610
9616,44.925908,-122.982753
9617,43.029559,-89.397796


In [ ]:
dfwithlatlong = df[["lat", "long"]]
mm_scaler = MinMaxScaler()
scaled_values = mm_scaler.fit_transform(dfwithlatlong)
scaled_columns = ["lat_mm", "long_mm"] #эти колонки будут обозначать значения, которые приобразованы минмакс скейлером
mm_scaled_df = pd.DataFrame(scaled_values, columns=scaled_columns)
df = pd.concat([df.reset_index(drop=True), mm_scaled_df], axis=1)#доабвляю в исходный датасет
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,age_category_old,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,lat_mm,long_mm
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,0.341769,0.288699
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,0.468665,0.333055
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.313561,0.292022
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,0.356215,0.314844
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,0.361810,0.293893


In [22]:
#для наглядности сделаю иначе
geo = df[["lat","lat_std","lat_mm", "long", "long_std", "long_mm"]]
geo
#значения будут разными std-standartscaler Стандартизирует: среднее = 0, std = 1 минмакс в свою очередь Масштабирует значения в заданный диапазон [0,1] по умолчанию

,lat,lat_std,lat_mm,long,long_std,long_mm
0,35.060000,-0.619172,0.341769,-85.250000,0.484245,0.288699
1,40.821805,0.388014,0.468665,-74.061962,1.110800,0.333055
2,33.779214,-0.843059,0.313561,-84.411811,0.531185,0.292022
3,35.715954,-0.504509,0.356215,-78.655304,0.853562,0.314844
4,35.970000,-0.460101,0.361810,-83.940000,0.557607,0.293893
...,...,...,...,...,...,...
9614,42.123900,0.615626,0.497341,-79.189500,0.823646,0.312727
9615,43.216990,0.806702,0.521415,-77.755610,0.903947,0.318411
9616,44.925908,1.105428,0.559052,-122.982753,-1.628875,0.139104
9617,43.029559,0.773939,0.517287,-89.397796,0.251959,0.272255


MinMaxScaler (x - min) / (max - min)
StandardScaler (x - mean) / std

MinMaxScaler: полезен для алгоритмов, которые чувствительны к масштабу, например KNN или нейросети.

StandardScaler: удобен для моделей, которые предполагают нормальное распределение признаков, например логистическая регрессия, SVM.

In [23]:
#выведу матрицу корреляции, чтоб показать, что значения остаются теми же 
geo.corr()

,lat,lat_std,lat_mm,long,long_std,long_mm
lat,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
lat_std,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
lat_mm,1.000000,1.000000,1.000000,-0.134596,-0.134596,-0.134596
long,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000
long_std,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000
long_mm,-0.134596,-0.134596,-0.134596,1.000000,1.000000,1.000000


In [25]:
print(df["posting_date"]) #признаки с датой, но оба приведены к типу: объект
print(df["date"])

0       2021-04-17T12:30:50-0400
1       2021-05-03T15:40:21-0400
2       2021-04-28T03:52:20-0700
3       2021-04-17T10:08:57-0400
4       2021-04-08T15:10:56-0400
                  ...           
9614    2021-04-10T16:33:57-0400
9615    2021-05-03T09:36:30-0400
9616    2021-04-22T12:14:01-0700
9617    2021-04-14T09:14:42-0500
9618    2021-04-24T13:50:49-0400
Name: posting_date, Length: 9619, dtype: object
0       2021-04-17 16:30:50+00:00
1       2021-05-03 19:40:21+00:00
2       2021-04-28 10:52:20+00:00
3       2021-04-17 14:08:57+00:00
4       2021-04-08 19:10:56+00:00
                  ...            
9614    2021-04-10 20:33:57+00:00
9615    2021-05-03 13:36:30+00:00
9616    2021-04-22 19:14:01+00:00
9617    2021-04-14 14:14:42+00:00
9618    2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: object


In [26]:
df["date"] = pd.to_datetime(df["date"])
df.date

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-04-08 19:10:56+00:00
                  ...           
9614   2021-04-10 20:33:57+00:00
9615   2021-05-03 13:36:30+00:00
9616   2021-04-22 19:14:01+00:00
9617   2021-04-14 14:14:42+00:00
9618   2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: datetime64[ns, UTC]

In [27]:
df["date"].dt.month
df["posting_date"] = pd.to_datetime(df["posting_date"],utc=True)
df["posting_date"].dt.month
df["month"] = df["posting_date"].dt.month
print(df["month"])

0       4
1       5
2       4
3       4
4       4
       ..
9614    4
9615    5
9616    4
9617    4
9618    4
Name: month, Length: 9619, dtype: int32


In [28]:
df["dayofweek"] = df["posting_date"].dt.day_name()
print(df["dayofweek"])

0        Saturday
1          Monday
2       Wednesday
3        Saturday
4        Thursday
          ...    
9614     Saturday
9615       Monday
9616     Thursday
9617    Wednesday
9618     Saturday
Name: dayofweek, Length: 9619, dtype: object


In [29]:
df["diff_years"] = df["year"] - df["posting_date"].dt.year 
df["diff_years"]

0       -1
1       -5
2       -4
3       -8
4       -9
        ..
9614   -19
9615   -13
9616   -10
9617    -6
9618    -4
Name: diff_years, Length: 9619, dtype: int64

In [30]:

std_scaler = StandardScaler()
std_scaler_col = std_scaler.fit_transform(df[["diff_years", "month"]])
scaled_columns = ["diff_years_std", "month_std"]
std_scaled_df = pd.DataFrame(std_scaler_col, columns=scaled_columns)
df = pd.concat([df.reset_index(drop=True), std_scaled_df], axis=1)


ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[["dayofweek"]])
ohe_cat = ohe.transform(df[["dayofweek"]])
ohe_columns = ohe.get_feature_names_out(["dayofweek"]).tolist()
ohe_df = pd.DataFrame(ohe_cat, columns=ohe_columns)
ohe_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df, ohe_df], axis=1)
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,diff_years,diff_years_std,month_std,dayofweek_Friday,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday,dayofweek_Wednesday
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,-1,1.322394,-0.615846,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,-5,0.695973,1.623784,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,-4,0.852578,-0.615846,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,-8,0.226157,-0.615846,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,-9,0.069552,-0.615846,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,7304876387,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2002,toyota,rav4,gas,150000,...,-19,-1.496501,-0.615846,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9615,7316152972,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,jeep,wrangler,gas,113573,...,-13,-0.556869,1.623784,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9616,7310993818,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,audi,a3 2.0t premium plus pzev,gas,150184,...,-10,-0.087054,-0.615846,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9617,7306637427,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,porsche,cayenne,hybrid,61943,...,-6,0.539367,-0.615846,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [31]:
columns_for_drop = [ 'url', 'region', 'region_url', 'manufacturer',
                     'fuel', 'odometer', 'title_status', 'transmission',
                    'image_url', 'description', 'state',
                    'odometer_km', 'odometer/price', 'manufacturer_model',
                    'desc_len', 'model_in_desc', 'price_k$',
                    'odometer/price_std']


df_prepared = df.drop(columns_for_drop, errors="ignore")

In [32]:
print(df.columns.tolist())
print(df.shape)

['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model', 'fuel', 'odometer', 'title_status', 'transmission', 'image_url', 'description', 'state', 'lat', 'long', 'posting_date', 'price_category', 'date', 'odometer_km', 'odometer/price', 'region_new', 'region_corrected', 'manufacturer_model', 'desc_len', 'model_in_desc', 'price_k$', 'age_category', 'model_len', 'model_word_count', 'is_audi', 'is_ford', 'is_chevrolet', 'is_toyota', 'is_jeep', 'is_honda', 'is_nissan', 'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid', 'x0_other', 'std_scaled_odometer', 'std_scaled_price', 'short_model', 'short_model_-benz', 'short_model_1', 'short_model_124', 'short_model_128i', 'short_model_135i', 'short_model_1500', 'short_model_163869', 'short_model_1928', 'short_model_1951', 'short_model_1954', 'short_model_1960s', 'short_model_1966', 'short_model_1968', 'short_model_1972', 'short_model_1977', 'short_model_1978', 'short_model_198.5', 'short_model_1988', 'short_model_1994', 'short

In [33]:
df.to_csv("data/vehicles_dataset_prepared.csv", index=False)